In [106]:
import json
from collections import defaultdict
import string
import re
import numpy as np
import math
import csv

In [107]:
with open('dev.json') as json_file:  
    data = json.load(json_file)

In [108]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [109]:
#print(data['data'][0]['paragraphs'][0]['qas'][0].keys())
#print(data['data'][0]['paragraphs'][0]['qas'][1]['orig_answer'])
#print(data['data'][0]['paragraphs'][0]['qas'][1]['answers'])
#print(data['data'][0]['paragraphs'][0]['qas'][1]['id'])

In [110]:
# print(len(data['data'][0]['paragraphs']))
num_paragraphs = len(data['data'])
distance_from_prev_question = {}
context_len_id = {}

In [111]:
yes_no = defaultdict(int)
qid_turn_num = defaultdict(int)
qid_ans_len = defaultdict(int)
qid_unans = {}
contextid_qids = defaultdict(list)
qid_to_contextid = defaultdict(int)
qid_to_oringal_ans = {}
total_ques = 0
for i in range(num_paragraphs):
    paragraphs = data['data'][i]['paragraphs']
    context = paragraphs[0]['context']
    context_len_id[i] = len(normalize_answer(context).split())

    qas = paragraphs[0]['qas']
    num_questions = len(qas)
    prev_question_start = 0
    question_ids = []
    prev_ans_text = ''
    for j in range(num_questions):
        total_ques += 1
        qid = qas[j]['id']
        question_ids.append(qid)
        qid_to_contextid[qid] = i
        question_text = qas[j]['question']
        original_ans = qas[j]['orig_answer']
        #qid_ans_len[qid] = normalize_answer(original_ans)
        yesno = qas[j]['yesno']
        
        yes_no[yesno] += 1
        answer_text = original_ans['text']
        qid_to_oringal_ans[qid] = answer_text
        if answer_text == 'CANNOTANSWER':
            qid_unans[qid] = 1
        qid_turn_num[qid] = j 
        qid_ans_len[qid] = len(normalize_answer(answer_text).split())
        
        if j == 0:
            distance_from_prev_question[qid] = -1
            prev_question_start = original_ans['answer_start']
            prev_ans_text = answer_text
        else:
            curr_question_start = original_ans['answer_start']
            distance_from_prev_question[qid] = math.fabs(curr_question_start - prev_question_start)
            prev_question_start = curr_question_start
            prev_ans_text = answer_text
    contextid_qids[i] = question_ids

In [112]:
print(len(distance_from_prev_question))
print(len(qid_unans))
print(len(qid_turn_num))
print(len(qid_ans_len))

7354
1486
7354
7354


In [113]:
import pickle
output = pickle.load(open("pred2.pckl", "rb" ))
output_old = pickle.load(open("pred2-old.pckl", "rb" ))

In [114]:
predictions = output['predictions']
no_ans_scores = output['no_ans_scores']

predictions_old = output_old['predictions']
no_ans_scores_old = output_old['no_ans_scores']

In [125]:
qid_to_f1_old = {}
unanswerable_old = {}
qid_to_ans_old = {}
count_old = 0
for i in range(len(predictions_old)):
    single_out = predictions_old[i]
    no_ans_score = no_ans_scores_old[i]
    qids = single_out['qid']
    F1s = single_out['F1']
    answers = single_out['best_span_str']
    for j in range(len(qids)):
        qid = qids[j]
        qid_to_f1_old[qid] = F1s[j]
        if qid_to_oringal_ans[qid] == 'CANNOTANSWER' and answers[j] != 'CANNOTANSWER':
            qid_to_f1_old[qid] = 0
        if qid_to_oringal_ans[qid] != 'CANNOTANSWER' and answers[j] == 'CANNOTANSWER':
            qid_to_f1_old[qid] = 0
        if answers[j] == 'CANNOTANSWER':
            unanswerable_old[qid] = 1
            count_old += 1
        else:
            unanswerable_old[qid] = 0
        qid_to_ans_old[qid] = answers[j]

In [126]:
print(count_old)

1712


In [127]:
qid_to_f1 = {}
unanswerable = {}
qid_to_ans = {}
count = 0
for i in range(len(predictions)):
    single_out = predictions[i]
    no_ans_score = no_ans_scores[i]
    qids = single_out['qid']
    F1s = single_out['F1']
    answers = single_out['best_span_str']
    for j in range(len(qids)):
        qid = qids[j]
        qid_to_f1[qid] = F1s[j]
        if qid_to_oringal_ans[qid] == 'CANNOTANSWER' and answers[j] != 'CANNOTANSWER':
            qid_to_f1[qid] = 0
        if qid_to_oringal_ans[qid] != 'CANNOTANSWER' and answers[j] == 'CANNOTANSWER':
            qid_to_f1[qid] = 0
        if answers[j] == 'CANNOTANSWER':
            unanswerable[qid] = 1
            count += 1
        else:
            unanswerable[qid] = 0
        qid_to_ans[qid] = answers[j]

In [128]:
num_0_f1_old = 0
num_100_f1_old = 0
for key in qid_to_f1_old:
    if qid_to_f1_old[key] == 0:
        num_0_f1_old += 1
    if qid_to_f1_old[key] == 100:
        num_100_f1_old += 1

In [129]:
num_0_f1 = 0
num_100_f1 = 0
for key in qid_to_f1:
    if qid_to_f1[key] == 0:
        num_0_f1 += 1
    if qid_to_f1[key] == 100:
        num_100_f1 += 1

In [130]:
print(num_0_f1_old)
print(num_100_f1_old)
print(num_0_f1)
print(num_100_f1)

1371
2449
1305
2066


In [131]:
# for key in unanswerable_old:
#     if unanswerable_old[key] == 0:
#         print(key)
wrongly_classified_ids = {}

In [132]:
TP_old = 0
FP_old = 0
TN_old = 0
FN_old = 0
mis_count_old = 0
for key in unanswerable_old.keys():
    if unanswerable_old[key] == 1 and key in qid_unans:
        TP_old += 1
    if unanswerable_old[key] == 0 and key in qid_unans:
        if qid_to_f1_old[key] != 0:
            # print(key)
            # print(qid_to_f1[key])
            qid_to_f1_old[key] = 0
            mis_count_old += 1
            wrongly_classified_ids[key] = 1
        FN_old += 1
    if unanswerable_old[key] == 1 and key not in qid_unans:
        if qid_to_f1_old[key] != 0:
            qid_to_f1_old[key] = 0
            # print(key)
            # print(qid_to_f1[key])
            mis_count_old += 1
            wrongly_classified_ids[key] = 1
        FP_old += 1
    if unanswerable_old[key] == 0 and key not in qid_unans:
        TN_old += 1

In [133]:
TP = 0
FP = 0
TN = 0
FN = 0
mis_count = 0
for key in unanswerable.keys():
    if unanswerable[key] == 1 and key in qid_unans:
        TP += 1
    if unanswerable[key] == 0 and key in qid_unans:
        if qid_to_f1[key] != 0:
            # print(key)
            # print(qid_to_f1[key])
            qid_to_f1[key] = 0
            mis_count += 1
            wrongly_classified_ids[key] = 1
        FN += 1
    if unanswerable[key] == 1 and key not in qid_unans:
        if qid_to_f1[key] != 0:
            qid_to_f1[key] = 0
            # print(key)
            # print(qid_to_f1[key])
            mis_count += 1
            wrongly_classified_ids[key] = 1
        FP += 1
    if unanswerable[key] == 0 and key not in qid_unans:
        TN += 1

In [134]:
print(mis_count_old)
print(mis_count)
print(len(wrongly_classified_ids))

0
0
0


In [135]:
num_0_f1_old = 0
num_100_f1_old = 0
for key in qid_to_f1_old:
    if qid_to_f1_old[key] == 0:
        num_0_f1_old += 1
    if qid_to_f1_old[key] == 100:
        num_100_f1_old += 1

In [136]:
num_0_f1 = 0
num_100_f1 = 0
for key in qid_to_f1:
    if qid_to_f1[key] == 0:
        num_0_f1 += 1
    if qid_to_f1[key] == 100:
        num_100_f1 += 1

In [137]:
print(num_0_f1_old)
print(num_100_f1_old)
print(num_0_f1)
print(num_100_f1)

1371
2449
1305
2066


In [249]:
import operator
sorted_qid_to_f1 = sorted(qid_to_f1.items(), key=operator.itemgetter(1))

In [250]:
print(len(sorted_qid_to_f1))

7354


In [251]:
count = 0
num_ques_f1 = defaultdict(int)
limit = 10
num_zeros = 0

for key in range(len(sorted_qid_to_f1)):

    if sorted_qid_to_f1[key][1] == 0:
        num_zeros += 1
    if sorted_qid_to_f1[key][1] > limit and sorted_qid_to_f1[key][1]  <= 2*limit:
        num_ques_f1[2*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 2 * limit and sorted_qid_to_f1[key][1]  <= 3*limit:
        num_ques_f1[3*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 3 * limit and sorted_qid_to_f1[key][1]  <= 4*limit:
        num_ques_f1[4*limit] += 1
    
    if sorted_qid_to_f1[key][1] > 4 *limit and sorted_qid_to_f1[key][1]  <= 5*limit:
        num_ques_f1[5*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 5*limit and sorted_qid_to_f1[key][1]  <= 6*limit:
        num_ques_f1[6*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 6*limit and sorted_qid_to_f1[key][1]  <= 7*limit:
        num_ques_f1[7*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 7*limit and sorted_qid_to_f1[key][1]  <= 8*limit:
        num_ques_f1[8*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 8*limit and sorted_qid_to_f1[key][1]  <= 9*limit:
        num_ques_f1[9*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 9*limit and sorted_qid_to_f1[key][1]  <= 10*limit:
        num_ques_f1[10*limit] += 1
print(num_ques_f1) 
print(num_zeros)
val = 0
for key in num_ques_f1.keys():
    val += num_ques_f1[key]
print(val)

defaultdict(<class 'int'>, {20: 630, 30: 297, 40: 131, 50: 120, 60: 199, 70: 279, 80: 458, 90: 693, 100: 2993})
1305
5800


In [ ]:
count = 0

for key in range(len(sorted_qid_to_f1)):
    if sorted_qid_to_f1[key][1] >= 15 and sorted_qid_to_f1[key][1] <=20 :
        num_ques_f1[limit] += 1
        count += 1
        qid = sorted_qid_to_f1[key][0]
        print('{} : {} : {} : {} : {}'.format(qid_to_contextid[qid], qid, qid_to_ans[qid], qid_to_oringal_ans[qid], sorted_qid_to_f1[key][1]))
        if count > 100:
            break

In [96]:
# Find those the old one found as answerable but not the new one
for key in qid_to_f1.keys():
    if key in qid_unans and key not in wrongly_classified_ids:
        if unanswerable_old[key] == 0 and qid_to_ans[key] == 'CANNOTANSWER':
            #if qid_to_f1_old[key] > 0:
            print(key)

C_3e7c4d0e33504faeb7704c042414bc79_1_q#2
C_55e01dd346874547983e428c1d6ff5ca_0_q#6
C_77b363670a744eafa2d37f4753ce535f_0_q#1
C_19f75daf1ddd412aa043fe06c0bdf89c_0_q#2
C_e3f3042742e443e59833b9d5f3f6c00d_1_q#3
C_982801502b2a47bab9da7247ee0db2d8_0_q#4


In [138]:
for key in qid_to_f1.keys():
    if qid_to_f1_old[key] < 50 and qid_to_f1[key] > 70:
            print(key)

C_0aaa843df0bd467b96e5a496fc0b033d_1_q#1
C_95a02857a40c472c9e6d8cb7b304fd2b_0_q#4
C_5ab583f64dbb47b995cf59328ea0af43_0_q#1
C_5ab583f64dbb47b995cf59328ea0af43_0_q#3
C_5ab583f64dbb47b995cf59328ea0af43_0_q#9
C_7c01702c75b94611becc862c15233ff9_1_q#1
C_01e52333288f47fcb1e611c5385ffe29_1_q#7
C_6abd2040a75d47168a9e4cca9ca3fed5_0_q#6
C_6abd2040a75d47168a9e4cca9ca3fed5_0_q#7
C_46fe62fdc8ca47ea96350dbaba1ec846_0_q#3
C_6e1f2e6127634211b25ec6356b3579f5_0_q#0
C_79838e4fce63478fae5bae6e38aff6c1_1_q#4
C_e6af1e489b6d4cc7962abc787e42f116_0_q#2
C_e6af1e489b6d4cc7962abc787e42f116_0_q#3
C_3b1946acb94349afadcf76ed7af2e227_0_q#2
C_3b1946acb94349afadcf76ed7af2e227_0_q#3
C_420bfcf5d8b344a480ac654f08ee55ad_0_q#7
C_cfc2c405e0c24c67b9b023faa38d72ba_0_q#10
C_bd83442cdece4bdf801b19836091eee6_0_q#3
C_28fc8af7d9f94c5e868770b55c3a16a0_0_q#4
C_0b63ef593dd04e27997b22d2c4496eae_0_q#0
C_14dca89984df455a86702f73591901fa_0_q#3
C_f0555dd820d84564a189474bbfffd4a1_0_q#1
C_f0555dd820d84564a189474bbfffd4a1_0_q#4
C_f0555dd820d84

In [147]:
curr_qid = 'C_fa3b87ef28ad4c8ebd4cda7534fd22ca_1_q#3'
cid = qid_to_contextid[curr_qid]
origanswer = qid_to_oringal_ans[curr_qid]

old_answer = qid_to_ans_old[curr_qid]
f1_old = qid_to_f1_old[curr_qid]
new_answer = qid_to_ans[curr_qid]
f1_new = qid_to_f1[curr_qid]

print('Context Id: ', cid)
print('Original Answer: ', origanswer)
print('Old Answer: ', old_answer)
print('New Answer: ', new_answer)
print('Old F1 Score: ', f1_old)
print('New F1 Score: ', f1_new)
print('Unanswerable according to old : ', unanswerable_old[curr_qid] == 1)
print('Unanswerable according to new : ', unanswerable[curr_qid] == 1)

Context Id:  44
Original Answer:  "Lions, Tigers & Bears"
Old Answer:  "Bust Your Windows"
New Answer:  "Lions, Tigers & Bears"
Old F1 Score:  0.0
New F1 Score:  100.0
Unanswerable according to old :  False
Unanswerable according to new :  False


In [145]:
data['data'][44]['paragraphs']

[{'context': 'Turner recorded his fifth studio album in Burbank, California, in October 2012. The reason for recording abroad the first time in California was that producer Rich Costey wouldn\'t come to England with all his equipment so he convinced Turner to record it in Burbank. Frank Turner announced via Twitter on 28 October 2012 that recording was completed, and mixing and mastering would be done in time for a March 2013 release date. During the November and December UK tour a free CD titled Good Hangs from Xtra Mile Recordings was distributed for free at Frank Turner shows, this included a yet-to-be-released Turner song titled "Tattoos" plus the Mongol Horde song "Casual Threats From Weekend Hardmen". On Christmas Day 2012, the song "Four Simple Words" was released on Xtra Mile Recordings\' website as a free download along with the b-side "Cowboy Chords". On 4 January 2013, a video was shot for what would be the first single. On the same day Frank announced on Twitter that the si